In [1]:
pip install crypto

Note: you may need to restart the kernel to use updated packages.


In [20]:
from datetime import datetime
import pytz
import json
#import crypto
#import sys
#sys.modules['Crypto'] = crypto
import pyrebase
import requests

# Access Firebase
with open("C:\\Users\\pcw\\Desktop\\3-2_class\\MP (Practice&Project[Team&Individual])\\sportify\\auth.json") as f:
    config = json.load(f)
firebase = pyrebase.initialize_app(config)
db = firebase.database()

# Function to get matches for a specific competition
def get_matches():
    url = "https://api.football-data.org/v4/competitions/PL/matches"
    headers = {
    "X-Auth-Token": "c889833ff3204c52adee15eb28d9bcc8",
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error getting standings. Status code: {response.status_code}")
        return None

def get_current_kst_date():
    kst = pytz.timezone('Asia/Seoul')
    now_utc = datetime.utcnow().replace(tzinfo=pytz.UTC)
    now_kst = now_utc.astimezone(kst)
    return now_kst.strftime('%m-%d %H:%M')

def get_current_utc_date():
    now = datetime.utcnow()
    return now.strftime('%Y-%m-%dT%H:%M:%SZ')
    
def convert_utc_to_kst(utc_time_str):
    utc_time = datetime.strptime(utc_time_str, '%Y-%m-%dT%H:%M:%SZ')
    kst = pytz.timezone('Asia/Seoul')
    kst_time = utc_time.replace(tzinfo=pytz.utc).astimezone(kst)
    return kst_time.strftime('%m-%d %H:%M')

team_name_mapping = {
    "Brighton & Hove Albion FC": "Brighton",
    "AFC Bournemouth": "Bournemouth",
    "Nottingham Forest FC": "Nottingham",
    "Sheffield United FC": "Sheffield",
    "Arsenal FC": "Arsenal",
    "Brentford FC": "Brentford",
    "Aston Villa FC": "Aston Villa",
    "Tottenham Hotspur FC": "Tottenham",
    "Manchester United FC": "Man United",
    "Everton FC": "Everton",
    "Wolverhampton Wanderers FC": "Wolverhampton",
    "Fulham FC": "Fulham",
    "West Ham United FC": "West Ham",
    "Burnley FC": "Burnley",
    "Manchester City FC": "Man City",
    "Liverpool FC": "Liverpool",
    "Crystal Palace FC": "Crystal Palace",
    "Luton Town FC": "Luton Town",
    "Newcastle United FC": "Newcastle",
    "Chelsea FC": "Chelsea"
}

def convert_team_names(team_name):
    # Use the team_name_mapping dictionary to get the abbreviated name
    return team_name_mapping.get(team_name, team_name)

# Set the headers with your API key
# Statistics 가져오는 API
headers = {
	"X-RapidAPI-Key": "2eb0ad9717msh5a898288a798d61p1803ffjsn8c47cfe397c2",
	"X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
}

url = "https://api.football-data.org/v4"

# Function to get fixture id to be used in get_statistics method
def get_fixtures(date):
    url = "https://api-football-v1.p.rapidapi.com/v3/fixtures"
    querystring = {"date":date,"league":"39","season":"2023"}
    headers = {
	"X-RapidAPI-Key": "2eb0ad9717msh5a898288a798d61p1803ffjsn8c47cfe397c2",
	"X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
    }
    response = requests.get(url, headers=headers, params=querystring)
    return response.json()

# Function to get match statistics
def get_statistics(id):
    url = "https://api-football-v1.p.rapidapi.com/v3/fixtures/statistics"
    querystring = {"fixture":id}
    headers = {
	"X-RapidAPI-Key": "2eb0ad9717msh5a898288a798d61p1803ffjsn8c47cfe397c2",
	"X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
    }
    response = requests.get(url, headers=headers, params=querystring)
    return response.json()

def convert_utc_to_kst_for_statistics(utc_time_str):
    utc_time = datetime.strptime(utc_time_str, '%Y-%m-%dT%H:%M:%S%z')
    kst = pytz.timezone('Asia/Seoul')
    kst_time = utc_time.replace(tzinfo=pytz.utc).astimezone(kst)
    return kst_time.strftime('%m-%d %H:%M')


# date list
date_list = []
home_list = []
away_list = []
statistics_list = []

# current UTC time
current_utc_date = datetime.utcnow()

# Fetch matches data
matches_data = get_matches()
if matches_data:
    # Extract the matches data from the response
    matches = matches_data['matches']
    # Filter out finished matches
    finished_matches = [match for match in matches if match["status"] == "FINISHED"]
    # finished_matches의 index를 거꾸로 하기.
    matches_sorted = sorted(finished_matches, key=lambda x: abs((datetime.strptime(x["utcDate"], '%Y-%m-%dT%H:%M:%SZ') - datetime.strptime(get_current_utc_date(), '%Y-%m-%dT%H:%M:%SZ')).total_seconds()), reverse=False)
    # Limit to the top 10 matches
    for match_info in matches_sorted[:30]:
        # For statistics data
        if (datetime.strptime(match_info["utcDate"], "%Y-%m-%dT%H:%M:%SZ") < current_utc_date):
            sample_date = match_info["utcDate"][0:10]
            date_list.append(sample_date)
            fixture_data = get_fixtures(sample_date)['response']
            for j in range(len(fixture_data)):
                fixture_id = str(fixture_data[j]['fixture']['id'])
                sample_time = convert_utc_to_kst_for_statistics(fixture_data[j]['fixture']['date'])
                
                statistics = get_statistics(fixture_id)
                statistics_data = {
                    "time" : sample_time,
                    "homeTeam" : fixture_data[j]['teams']['home']['name'],
                    "awayTeam" : fixture_data[j]['teams']['away']['name'],
                    f"{statistics['response'][0]['team']['name']}'s {statistics['response'][0]['statistics'][0]['type']}" : statistics['response'][0]['statistics'][0]['value'],
                    f"{statistics['response'][0]['team']['name']}'s {statistics['response'][0]['statistics'][1]['type']}" : statistics['response'][0]['statistics'][1]['value'],
                    f"{statistics['response'][0]['team']['name']}'s {statistics['response'][0]['statistics'][2]['type']}" : statistics['response'][0]['statistics'][2]['value'],
                    f"{statistics['response'][0]['team']['name']}'s {statistics['response'][0]['statistics'][3]['type']}" : statistics['response'][0]['statistics'][3]['value'],
                    f"{statistics['response'][0]['team']['name']}'s {statistics['response'][0]['statistics'][4]['type']}" : statistics['response'][0]['statistics'][4]['value'],
                    f"{statistics['response'][0]['team']['name']}'s {statistics['response'][0]['statistics'][5]['type']}" : statistics['response'][0]['statistics'][5]['value'],
                    f"{statistics['response'][0]['team']['name']}'s {statistics['response'][0]['statistics'][6]['type']}" : statistics['response'][0]['statistics'][6]['value'],
                    f"{statistics['response'][0]['team']['name']}'s {statistics['response'][0]['statistics'][7]['type']}" : statistics['response'][0]['statistics'][7]['value'],
                    f"{statistics['response'][0]['team']['name']}'s {statistics['response'][0]['statistics'][8]['type']}" : statistics['response'][0]['statistics'][8]['value'],
                    f"{statistics['response'][0]['team']['name']}'s {statistics['response'][0]['statistics'][9]['type']}" : statistics['response'][0]['statistics'][9]['value'],
                    f"{statistics['response'][0]['team']['name']}'s {statistics['response'][0]['statistics'][10]['type']}" : statistics['response'][0]['statistics'][10]['value'],
                    f"{statistics['response'][0]['team']['name']}'s {statistics['response'][0]['statistics'][11]['type']}" : statistics['response'][0]['statistics'][11]['value'],
                    f"{statistics['response'][0]['team']['name']}'s {statistics['response'][0]['statistics'][12]['type']}" : statistics['response'][0]['statistics'][12]['value'],
                    f"{statistics['response'][0]['team']['name']}'s {statistics['response'][0]['statistics'][13]['type']}" : statistics['response'][0]['statistics'][13]['value'],
                    f"{statistics['response'][0]['team']['name']}'s {statistics['response'][0]['statistics'][14]['type']}" : statistics['response'][0]['statistics'][14]['value'],
                    f"{statistics['response'][0]['team']['name']}'s {statistics['response'][0]['statistics'][15]['type']}" : statistics['response'][0]['statistics'][15]['value'],
                    f"{statistics['response'][0]['team']['name']}'s {statistics['response'][0]['statistics'][16]['type']}" : statistics['response'][0]['statistics'][16]['value'],
                    f"{statistics['response'][1]['team']['name']}'s {statistics['response'][1]['statistics'][0]['type']}" : statistics['response'][1]['statistics'][0]['value'],
                    f"{statistics['response'][1]['team']['name']}'s {statistics['response'][1]['statistics'][1]['type']}" : statistics['response'][1]['statistics'][1]['value'],
                    f"{statistics['response'][1]['team']['name']}'s {statistics['response'][1]['statistics'][2]['type']}" : statistics['response'][1]['statistics'][2]['value'],
                    f"{statistics['response'][1]['team']['name']}'s {statistics['response'][1]['statistics'][3]['type']}" : statistics['response'][1]['statistics'][3]['value'],
                    f"{statistics['response'][1]['team']['name']}'s {statistics['response'][1]['statistics'][4]['type']}" : statistics['response'][1]['statistics'][4]['value'],
                    f"{statistics['response'][1]['team']['name']}'s {statistics['response'][1]['statistics'][5]['type']}" : statistics['response'][1]['statistics'][5]['value'],
                    f"{statistics['response'][1]['team']['name']}'s {statistics['response'][1]['statistics'][6]['type']}" : statistics['response'][1]['statistics'][6]['value'],
                    f"{statistics['response'][1]['team']['name']}'s {statistics['response'][1]['statistics'][7]['type']}" : statistics['response'][1]['statistics'][7]['value'],
                    f"{statistics['response'][1]['team']['name']}'s {statistics['response'][1]['statistics'][8]['type']}" : statistics['response'][1]['statistics'][8]['value'],
                    f"{statistics['response'][1]['team']['name']}'s {statistics['response'][1]['statistics'][9]['type']}" : statistics['response'][1]['statistics'][9]['value'],
                    f"{statistics['response'][1]['team']['name']}'s {statistics['response'][1]['statistics'][10]['type']}" : statistics['response'][1]['statistics'][10]['value'],
                    f"{statistics['response'][1]['team']['name']}'s {statistics['response'][1]['statistics'][11]['type']}" : statistics['response'][1]['statistics'][11]['value'],
                    f"{statistics['response'][1]['team']['name']}'s {statistics['response'][1]['statistics'][12]['type']}" : statistics['response'][1]['statistics'][12]['value'],
                    f"{statistics['response'][1]['team']['name']}'s {statistics['response'][1]['statistics'][13]['type']}" : statistics['response'][1]['statistics'][13]['value'],
                    f"{statistics['response'][1]['team']['name']}'s {statistics['response'][1]['statistics'][14]['type']}" : statistics['response'][1]['statistics'][14]['value'],
                    f"{statistics['response'][1]['team']['name']}'s {statistics['response'][1]['statistics'][15]['type']}" : statistics['response'][1]['statistics'][15]['value'],
                    f"{statistics['response'][1]['team']['name']}'s {statistics['response'][1]['statistics'][16]['type']}" : statistics['response'][1]['statistics'][16]['value'],
                }
                statistics_list.append(statistics_data)
    # Remove duplicated elements
    statistics_list = list({str(statistics_dict): statistics_dict for statistics_dict in statistics_list}.values())
    # Push the transformed statistics data to Firebase
    db.child("statistics").set(statistics_list)
    print("Top 10 matches data saved to Firebase successfully")


KeyError: 'response'

# Get Fixture ID from "V3 - Fixtures"

In [77]:
matches_data['response'][0]

{'fixture': {'id': 1035297,
  'referee': 'M. Salisbury',
  'timezone': 'UTC',
  'date': '2023-11-27T20:00:00+00:00',
  'timestamp': 1701115200,
  'periods': {'first': 1701115200, 'second': 1701118800},
  'venue': {'id': 535, 'name': 'Craven Cottage', 'city': 'London'},
  'status': {'long': 'Match Finished', 'short': 'FT', 'elapsed': 90}},
 'league': {'id': 39,
  'name': 'Premier League',
  'country': 'England',
  'logo': 'https://media-4.api-sports.io/football/leagues/39.png',
  'flag': 'https://media-4.api-sports.io/flags/gb.svg',
  'season': 2023,
  'round': 'Regular Season - 13'},
 'teams': {'home': {'id': 36,
   'name': 'Fulham',
   'logo': 'https://media-4.api-sports.io/football/teams/36.png',
   'winner': True},
  'away': {'id': 39,
   'name': 'Wolves',
   'logo': 'https://media-4.api-sports.io/football/teams/39.png',
   'winner': False}},
 'goals': {'home': 3, 'away': 2},
 'score': {'halftime': {'home': 1, 'away': 1},
  'fulltime': {'home': 3, 'away': 2},
  'extratime': {'home':

# Premier League ID from "V3 - Search League" 

In [46]:
print(matches_data['response'][5])

{'league': {'id': 39, 'name': 'Premier League', 'type': 'League', 'logo': 'https://media-4.api-sports.io/football/leagues/39.png'}, 'country': {'name': 'England', 'code': 'GB', 'flag': 'https://media-4.api-sports.io/flags/gb.svg'}, 'seasons': [{'year': 2010, 'start': '2010-08-14', 'end': '2011-05-17', 'current': False, 'coverage': {'fixtures': {'events': True, 'lineups': True, 'statistics_fixtures': False, 'statistics_players': False}, 'standings': True, 'players': True, 'top_scorers': True, 'top_assists': True, 'top_cards': True, 'injuries': False, 'predictions': True, 'odds': False}}, {'year': 2011, 'start': '2011-08-13', 'end': '2012-05-13', 'current': False, 'coverage': {'fixtures': {'events': True, 'lineups': True, 'statistics_fixtures': False, 'statistics_players': False}, 'standings': True, 'players': True, 'top_scorers': True, 'top_assists': True, 'top_cards': True, 'injuries': False, 'predictions': True, 'odds': False}}, {'year': 2012, 'start': '2012-08-18', 'end': '2013-05-19

# Team ID from "V3 - Search Team"

In [73]:
for i in range(20):
    print(f"{i}번째:",matches_data['response'][i])

0번째: {'team': {'id': 33, 'name': 'Manchester United', 'code': 'MUN', 'country': 'England', 'founded': 1878, 'national': False, 'logo': 'https://media-4.api-sports.io/football/teams/33.png'}, 'venue': {'id': 556, 'name': 'Old Trafford', 'address': 'Sir Matt Busby Way', 'city': 'Manchester', 'capacity': 76212, 'surface': 'grass', 'image': 'https://media-4.api-sports.io/football/venues/556.png'}}
1번째: {'team': {'id': 34, 'name': 'Newcastle', 'code': 'NEW', 'country': 'England', 'founded': 1892, 'national': False, 'logo': 'https://media-4.api-sports.io/football/teams/34.png'}, 'venue': {'id': 562, 'name': "St. James' Park", 'address': 'St. James&apos; Street', 'city': 'Newcastle upon Tyne', 'capacity': 52758, 'surface': 'grass', 'image': 'https://media-4.api-sports.io/football/venues/562.png'}}
2번째: {'team': {'id': 35, 'name': 'Bournemouth', 'code': 'BOU', 'country': 'England', 'founded': 1899, 'national': False, 'logo': 'https://media-4.api-sports.io/football/teams/35.png'}, 'venue': {'id